# Public Key Infrastructure (PKI) and Certificates

## Contents
1. PKI Role in IT Security
2. Creating Self-Signed Certificate
3. Certificate Components
4. Significance for Secure Communication

## Prerequisites:

```bash
pip install cryptography
```

In [1]:
from cryptography import x509
from cryptography.x509.oid import NameOID, ExtensionOID
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.backends import default_backend
import datetime
import ipaddress

## 1. PKI Role in IT Security

**PKI (Public Key Infrastructure)** = Framework for secure digital communication

### Core Components:
- **Certificate Authority (CA):** Issues and signs certificates
- **Registration Authority (RA):** Verifies certificate requests
- **Digital Certificates:** Bind public keys to identities
- **Certificate Repository:** Stores and distributes certificates
- **Certificate Revocation List (CRL):** Lists revoked certificates

### Main Functions:
1. **Authentication:** Verify identity of parties
2. **Encryption:** Secure data transmission
3. **Integrity:** Ensure data not modified
4. **Non-repudiation:** Prove origin of communication

### Use Cases:
- HTTPS/TLS (websites)
- Email signing (S/MIME)
- Code signing
- VPN authentication
- Document signing

## 2. Creating Self-Signed Certificate

### Certificate Types:
- **Self-Signed:** Created and signed by same entity (testing/internal)
- **CA-Signed:** Signed by trusted CA (Let's Encrypt, DigiCert) (production)

In [2]:
# Generate private key
print("Generating RSA private key (2048 bits)...")
private_key = rsa.generate_private_key(
    public_exponent=65537, key_size=2048, backend=default_backend()
)
print("Private key generated")

# Create subject (certificate owner info)
subject = issuer = x509.Name(
    [
        x509.NameAttribute(NameOID.COUNTRY_NAME, "AT"),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, "Niederösterreich"),
        x509.NameAttribute(NameOID.LOCALITY_NAME, "Wiener Neustadt"),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, "Awesome Dev Company"),
        x509.NameAttribute(NameOID.ORGANIZATIONAL_UNIT_NAME, "IT Security"),
        x509.NameAttribute(NameOID.COMMON_NAME, "example.com"),
    ]
)

print("\nCertificate subject created:")
for attr in subject:
    print(f"  {attr.oid._name}: {attr.value}")

Generating RSA private key (2048 bits)...
Private key generated

Certificate subject created:
  countryName: AT
  stateOrProvinceName: Niederösterreich
  localityName: Wiener Neustadt
  organizationName: Awesome Dev Company
  organizationalUnitName: IT Security
  commonName: example.com


In [3]:
# Build certificate
cert = (
    x509.CertificateBuilder()
    .subject_name(subject)
    .issuer_name(
        issuer  # Self-signed: subject = issuer
    )
    .public_key(private_key.public_key())
    .serial_number(x509.random_serial_number())
    .not_valid_before(datetime.datetime.utcnow())
    .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))
    .add_extension(
        x509.SubjectAlternativeName(
            [
                x509.DNSName("example.com"),
                x509.DNSName("www.example.com"),
                x509.DNSName("localhost"),
                x509.IPAddress(ipaddress.IPv4Address("127.0.0.1")),
            ]
        ),
        critical=False,
    )
    .add_extension(
        x509.BasicConstraints(ca=False, path_length=None),
        critical=True,
    )
    .add_extension(
        x509.KeyUsage(
            digital_signature=True,
            key_encipherment=True,
            content_commitment=False,
            data_encipherment=False,
            key_agreement=False,
            key_cert_sign=False,
            crl_sign=False,
            encipher_only=False,
            decipher_only=False,
        ),
        critical=True,
    )
    .add_extension(
        x509.ExtendedKeyUsage(
            [
                x509.oid.ExtendedKeyUsageOID.SERVER_AUTH,
                x509.oid.ExtendedKeyUsageOID.CLIENT_AUTH,
            ]
        ),
        critical=False,
    )
    .sign(private_key, hashes.SHA256(), default_backend())
)

print("Certificate created and self-signed")

Certificate created and self-signed


C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\1076450265.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  .not_valid_before(datetime.datetime.utcnow())
C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\1076450265.py:11: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  .not_valid_after(datetime.datetime.utcnow() + datetime.timedelta(days=365))


In [4]:
# Save certificate and private key
cert_pem = cert.public_bytes(serialization.Encoding.PEM)
key_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption(),
)

with open("certificate.crt", "wb") as f:
    f.write(cert_pem)
with open("private_key.key", "wb") as f:
    f.write(key_pem)

print("Saved to certificate.crt and private_key.key")
print(f"\nCertificate (CRT format):\n{cert_pem.decode()[:400]}...")

Saved to certificate.crt and private_key.key

Certificate (CRT format):
-----BEGIN CERTIFICATE-----
MIIEIzCCAwugAwIBAgIUar6Q0vAu5RYMk6CN/LjUe1LKAcgwDQYJKoZIhvcNAQEL
BQAwgY0xCzAJBgNVBAYTAkFUMRowGAYDVQQIDBFOaWVkZXLDtnN0ZXJyZWljaDEY
MBYGA1UEBwwPV2llbmVyIE5ldXN0YWR0MRwwGgYDVQQKDBNBd2Vzb21lIERldiBD
b21wYW55MRQwEgYDVQQLDAtJVCBTZWN1cml0eTEUMBIGA1UEAwwLZXhhbXBsZS5j
b20wHhcNMjUxMTA4MTA1NjE0WhcNMjYxMTA4MTA1NjE0WjCBjTELMAkGA1UEBhMC
QVQxGjAYBgNVBAgMEU5pZWRlcsO2c3RlcnJlaWNoMRgwFgY...


## 3. Certificate Components

### Required Components (X.509 Standard):

In [5]:
print("CERTIFICATE COMPONENTS")
print("=" * 70)

# Version
print(f"\n1. Version: {cert.version.name}")
print("   Purpose: Certificate format version (v3 is current standard)")

# Serial Number
print(f"\n2. Serial Number: {cert.serial_number}")
print("   Purpose: Unique identifier for certificate")

# Signature Algorithm
print(f"\n3. Signature Algorithm: {cert.signature_algorithm_oid._name}")
print("   Purpose: Algorithm used to sign certificate (SHA256withRSA)")

# Issuer
print(f"\n4. Issuer: {cert.issuer.rfc4514_string()}")
print("   Purpose: Who signed/issued the certificate (CA or self)")

# Validity Period
print("\n5. Validity Period:")
print(f"   Not Before: {cert.not_valid_before}")
print(f"   Not After: {cert.not_valid_after}")
print("   Purpose: Certificate lifetime (valid for 365 days)")

# Subject
print(f"\n6. Subject: {cert.subject.rfc4514_string()}")
print("   Purpose: Certificate owner identity")

# Public Key
print("\n7. Public Key Info:")
print("   Algorithm: RSA")
print(f"   Key Size: {private_key.key_size} bits")
print("   Purpose: Public key for encryption/verification")

# Extensions
print(f"\n8. Extensions ({len(cert.extensions)} total):")
for ext in cert.extensions:
    print(f"   - {ext.oid._name} (Critical: {ext.critical})")

CERTIFICATE COMPONENTS

1. Version: v3
   Purpose: Certificate format version (v3 is current standard)

2. Serial Number: 609402779242777245900782992193931863733098381768
   Purpose: Unique identifier for certificate

3. Signature Algorithm: sha256WithRSAEncryption
   Purpose: Algorithm used to sign certificate (SHA256withRSA)

4. Issuer: CN=example.com,OU=IT Security,O=Awesome Dev Company,L=Wiener Neustadt,ST=Niederösterreich,C=AT
   Purpose: Who signed/issued the certificate (CA or self)

5. Validity Period:
   Not Before: 2025-11-08 10:56:14
   Not After: 2026-11-08 10:56:14
   Purpose: Certificate lifetime (valid for 365 days)

6. Subject: CN=example.com,OU=IT Security,O=Awesome Dev Company,L=Wiener Neustadt,ST=Niederösterreich,C=AT
   Purpose: Certificate owner identity

7. Public Key Info:
   Algorithm: RSA
   Key Size: 2048 bits
   Purpose: Public key for encryption/verification

8. Extensions (4 total):
   - subjectAltName (Critical: False)
   - basicConstraints (Critical: True

C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\4035207306.py:22: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_before_utc.
  print(f"   Not Before: {cert.not_valid_before}")
C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\4035207306.py:23: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  print(f"   Not After: {cert.not_valid_after}")


In [6]:
# Detailed extension analysis
print("\nDETAILED EXTENSIONS")
print("=" * 70)

# Subject Alternative Name (SAN)
san = cert.extensions.get_extension_for_oid(ExtensionOID.SUBJECT_ALTERNATIVE_NAME)
print("\nSubject Alternative Name (SAN):")
for name in san.value:
    print(f"  - {name}")
print("  Purpose: Additional hostnames/IPs certificate is valid for")

# Basic Constraints
basic = cert.extensions.get_extension_for_oid(ExtensionOID.BASIC_CONSTRAINTS)
print("\nBasic Constraints:")
print(f"  CA: {basic.value.ca}")
print("  Purpose: Indicates if certificate can sign other certificates")

# Key Usage
key_usage = cert.extensions.get_extension_for_oid(ExtensionOID.KEY_USAGE)
print("\nKey Usage:")
print(f"  Digital Signature: {key_usage.value.digital_signature}")
print(f"  Key Encipherment: {key_usage.value.key_encipherment}")
print("  Purpose: Permitted cryptographic operations")

# Extended Key Usage
ext_usage = cert.extensions.get_extension_for_oid(ExtensionOID.EXTENDED_KEY_USAGE)
print("\nExtended Key Usage:")
for usage in ext_usage.value:
    print(f"  - {usage._name}")
print("  Purpose: Specific purposes certificate can be used for")


DETAILED EXTENSIONS

Subject Alternative Name (SAN):
  - <DNSName(value='example.com')>
  - <DNSName(value='www.example.com')>
  - <DNSName(value='localhost')>
  - <IPAddress(value=127.0.0.1)>
  Purpose: Additional hostnames/IPs certificate is valid for

Basic Constraints:
  CA: False
  Purpose: Indicates if certificate can sign other certificates

Key Usage:
  Digital Signature: True
  Key Encipherment: True
  Purpose: Permitted cryptographic operations

Extended Key Usage:
  - serverAuth
  - clientAuth
  Purpose: Specific purposes certificate can be used for


## 4. Significance for Secure Communication

### Component Significance:

| Component | Security Significance |
|-----------|----------------------|
| **Serial Number** | Uniquely identifies cert; used for revocation |
| **Issuer** | Establishes chain of trust; validates authenticity |
| **Subject** | Identifies certificate owner; prevents impersonation |
| **Public Key** | Enables encryption & signature verification |
| **Validity Period** | Limits exposure window; forces key rotation |
| **Signature** | Proves certificate hasn't been tampered with |
| **SAN** | Specifies valid domains/IPs; prevents misuse |
| **Key Usage** | Restricts operations; principle of least privilege |
| **Basic Constraints** | Prevents unauthorized CA certificates |

### Certificate Validation Process:

In [7]:
# Demonstrate certificate validation
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding

print("CERTIFICATE VALIDATION DEMO")
print("=" * 70)

# 1. Check validity period
now = datetime.datetime.utcnow()
valid_time = cert.not_valid_before <= now <= cert.not_valid_after
print(f"\n1. Time Validity: {valid_time}")
print(f"   Current: {now}")
print(f"   Valid: {cert.not_valid_before} to {cert.not_valid_after}")

# 2. Verify signature (self-signed)
public_key = cert.public_key()
try:
    public_key.verify(
        cert.signature,
        cert.tbs_certificate_bytes,
        asym_padding.PKCS1v15(),
        cert.signature_hash_algorithm,
    )
    sig_valid = True
except Exception:
    sig_valid = False

print(f"\n2. Signature Verification: {sig_valid}")
print("   Purpose: Ensures certificate not tampered with")

# 3. Check hostname match (simulated)
requested_host = "example.com"
san_names = [str(name.value) for name in san.value if isinstance(name, x509.DNSName)]
hostname_match = requested_host in san_names
print(f"\n3. Hostname Match: {hostname_match}")
print(f"   Requested: {requested_host}")
print(f"   Allowed: {san_names}")

# 4. Check key usage
server_auth_ok = x509.oid.ExtendedKeyUsageOID.SERVER_AUTH in ext_usage.value
print(f"\n4. Server Authentication Allowed: {server_auth_ok}")

msg = (
    "\nOverall: Certificate would be "
    f"{'ACCEPTED' if all([valid_time, sig_valid, hostname_match, server_auth_ok]) else 'REJECTED'}"
)

print(msg)

CERTIFICATE VALIDATION DEMO

1. Time Validity: True
   Current: 2025-11-08 10:56:14.519386
   Valid: 2025-11-08 10:56:14 to 2026-11-08 10:56:14

2. Signature Verification: True
   Purpose: Ensures certificate not tampered with

3. Hostname Match: True
   Requested: example.com
   Allowed: ['example.com', 'www.example.com', 'localhost']

4. Server Authentication Allowed: True

Overall: Certificate would be ACCEPTED


C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\1438737538.py:8: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\1438737538.py:9: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_before_utc.
  valid_time = cert.not_valid_before <= now <= cert.not_valid_after
C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\1438737538.py:9: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  valid_time = cert.not_valid_before <= now <= cert.not_valid_after
C:\Users\MarcelTurobin-Ort\AppData\Local\Temp\ipykernel_33660\1438737538.py:12: CryptographyDeprecationWarning

### TLS/HTTPS Handshake with Certificates:

In [8]:
print("TLS HANDSHAKE PROCESS (with Certificates)")
print("=" * 70)
print("""
1. Client Hello
   → Client initiates connection with supported cipher suites

2. Server Hello + Certificate
   → Server sends its certificate (includes public key)
   → Server chooses cipher suite

3. Client Validates Certificate:
   Check signature (issued by trusted CA?)
   Check validity period (expired?)
   Check hostname (matches requested domain?)
   Check revocation status (CRL/OCSP)
   Check key usage (allowed for TLS?)

4. Key Exchange:
   → Client generates session key
   → Encrypts session key with server's public key (from cert)
   → Sends encrypted session key to server

5. Server Decrypts:
   → Uses private key to decrypt session key

6. Secure Communication:
   → Both use session key for symmetric encryption (fast)
   → Certificate ensured: authenticity, encryption, integrity
""")

print("\nSECURITY PROVIDED BY CERTIFICATE:")
print("  Authentication: Server is who it claims to be")
print("  Encryption: Public key enables secure key exchange")
print("  Integrity: Signature proves certificate not modified")
print("  Trust: CA signature establishes trust chain")

TLS HANDSHAKE PROCESS (with Certificates)

1. Client Hello
   → Client initiates connection with supported cipher suites

2. Server Hello + Certificate
   → Server sends its certificate (includes public key)
   → Server chooses cipher suite

3. Client Validates Certificate:
   Check signature (issued by trusted CA?)
   Check validity period (expired?)
   Check hostname (matches requested domain?)
   Check revocation status (CRL/OCSP)
   Check key usage (allowed for TLS?)

4. Key Exchange:
   → Client generates session key
   → Encrypts session key with server's public key (from cert)
   → Sends encrypted session key to server

5. Server Decrypts:
   → Uses private key to decrypt session key

6. Secure Communication:
   → Both use session key for symmetric encryption (fast)
   → Certificate ensured: authenticity, encryption, integrity


SECURITY PROVIDED BY CERTIFICATE:
  Authentication: Server is who it claims to be
  Encryption: Public key enables secure key exchange
  Integrity: Sign

## 5. Self-Signed vs CA-Signed Certificates

In [9]:
print("CERTIFICATE COMPARISON")
print("=" * 70)

comparison = [
    ("Trust", "Not trusted by browsers", "Trusted globally"),
    ("Cost", "Free", "Free (Let's Encrypt) or paid"),
    ("Validation", "None", "Domain/Organization/Extended"),
    ("Use Case", "Development, internal", "Production, public sites"),
    ("Warnings", "Browser warning", "No warning"),
    ("Revocation", "Not possible", "CRL/OCSP supported"),
    ("Chain of Trust", "Self (no chain)", "CA → Root CA"),
]

print(f"\n{'Feature':<15} {'Self-Signed':<30} {"CA-Signed (e.g., Let's Encrypt)"}")
print("-" * 70)
for feature, self_signed, ca_signed in comparison:
    print(f"{feature:<15} {self_signed:<30} {ca_signed}")

CERTIFICATE COMPARISON

Feature         Self-Signed                    CA-Signed (e.g., Let's Encrypt)
----------------------------------------------------------------------
Trust           Not trusted by browsers        Trusted globally
Cost            Free                           Free (Let's Encrypt) or paid
Validation      None                           Domain/Organization/Extended
Use Case        Development, internal          Production, public sites
Warnings        Browser warning                No warning
Revocation      Not possible                   CRL/OCSP supported
Chain of Trust  Self (no chain)                CA → Root CA


### Let's Encrypt Process (Automated CA):

```bash
# Install certbot
sudo apt-get install certbot

# Obtain certificate (automatic domain validation)
sudo certbot certonly --standalone -d example.com

# Certificate stored in: /etc/letsencrypt/live/example.com/
# - fullchain.pem (certificate + chain)
# - privkey.pem (private key)
# - cert.pem (certificate only)

# Auto-renewal (Let's Encrypt certs expire after 90 days)
sudo certbot renew
```

**Let's Encrypt validates domain ownership via:**
- HTTP-01: Places file on web server
- DNS-01: Creates DNS TXT record
- TLS-ALPN-01: TLS handshake verification

## Summary

### PKI Role:
- Framework for managing digital certificates and public keys
- Enables authentication, encryption, integrity, non-repudiation
- Foundation of secure internet communication (HTTPS)

### Certificate Components (X.509):
1. **Version** - Format version
2. **Serial Number** - Unique ID for revocation
3. **Signature Algorithm** - How certificate is signed
4. **Issuer** - Who signed it (CA or self)
5. **Validity** - Lifetime window
6. **Subject** - Owner identity
7. **Public Key** - For encryption/verification
8. **Extensions** - SAN, key usage, constraints

### Significance:
- **Authentication:** Proves server identity
- **Encryption:** Public key enables secure key exchange
- **Integrity:** Signature prevents tampering
- **Trust Chain:** CA signature establishes trust

### Best Practices:
- Use CA-signed certs for production (Let's Encrypt is free)
- Self-signed only for development/testing
- Keep private keys secure
- Monitor expiration dates
- Use strong key sizes (2048+ bits)
- Implement certificate pinning for critical apps